In [16]:
from autogen.coding import CodeBlock
from autogen.coding.jupyter import DockerJupyterServer, JupyterCodeExecutor

with DockerJupyterServer() as server:
    executor = JupyterCodeExecutor(server)
    print(
        executor.execute_code_blocks(
            code_blocks=[
                CodeBlock(language="python", code="print('Hello, World!')"),
            ]
        )
    )

exit_code=0 output='Hello, World!\n' output_files=[]


In [21]:
from pathlib import Path

from autogen import ConversableAgent
from autogen.coding.jupyter import DockerJupyterServer, JupyterCodeExecutor

server = DockerJupyterServer()

output_dir = Path("coding")
output_dir.mkdir(exist_ok=True)

code_executor_agent = ConversableAgent(
    name="code_executor_agent",
    llm_config=False,
    code_execution_config={
        "executor": JupyterCodeExecutor(server, output_dir=output_dir),
    },
    human_input_mode="NEVER",
)

In [18]:
# The code writer agent's system message is to instruct the LLM on how to
# use the Jupyter code executor with IPython kernel.
import os
from dotenv import load_dotenv

load_dotenv()

llm_endpoint = os.environ["OLLAMA_ENDPOINT"]
model_name = os.environ["LLM_MODEL_NAME"]

api_key = os.environ["LLM_API_KEY"]

code_writer_system_message = """
You have been given coding capability to solve tasks using Python code in a stateful IPython kernel.
You are responsible for writing the code, and the user is responsible for executing the code.

When you write Python code, put the code in a markdown code block with the language set to Python.
For example:
```python
x = 3
```
You can use the variable `x` in subsequent code blocks.
```python
print(x)
```

Write code incrementally and leverage the statefulness of the kernel to avoid repeating code.
Import libraries in a separate code block.
Define a function or a class in a separate code block.
Run code that produces output in a separate code block.
Run code that involves expensive operations like download, upload, and call external APIs in a separate code block.

When your code produces an output, the output will be returned to you.
Because you have limited conversation memory, if your code creates an image,
the output will be a path to the image instead of the image itself."""

import os

code_writer_agent = ConversableAgent(
    "code_writer",
    system_message=code_writer_system_message,
    llm_config={"config_list": [
        {
        "model": model_name,
        "temperature": 0.7,
        "api_key": api_key,
        "base_url": llm_endpoint,
        "api_type": "openai",
        "price": [0.0, 0.0],
        }
    ]},
    code_execution_config=False,  # Turn off code execution for this agent.
    max_consecutive_auto_reply=2,
    human_input_mode="NEVER",
)

In [ ]:
import pprint

chat_result = code_executor_agent.initiate_chat(
code_writer_agent, message="Write Python code to calculate the 2nd Fibonacci number.",     
)

pprint.pprint(chat_result)

code_executor_agent (to code_writer):

Write Python code to calculate the 2nd Fibonacci number.

--------------------------------------------------------------------------------
code_writer (to code_executor_agent):

```python
# Define variables
a = 0
b = 1

# Calculate the 2nd Fibonacci number
f_2 = b

print(f_2)
```

This will output: `1`

--------------------------------------------------------------------------------

>>>>>>>> EXECUTING CODE BLOCK (inferred language is python)...
code_executor_agent (to code_writer):

exitcode: 0 (execution succeeded)
Code output: 1


--------------------------------------------------------------------------------
code_writer (to code_executor_agent):

The code I provided is correct. The 2nd Fibonacci number is indeed 1. 

If you want to calculate the nth Fibonacci number, we can modify the code as follows:

```python
# Define a function to calculate the nth Fibonacci number
def fibonacci(n):
    if n <= 0:
        return "Input should be a positiv

BrokenPipeError: [Errno 32] Broken pipe